In [1]:
%pip install -q ultralytics kagglehub pyyaml opencv-python efficientnet_pytorch torch torchvision

import os
import shutil
import pathlib
import yaml
import cv2
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
import torchvision.transforms as T
from torchvision.datasets import ImageFolder

import kagglehub
from ultralytics import YOLO
from efficientnet_pytorch import EfficientNet

#**********************************************
base_p = pathlib.Path("/content") if os.path.isdir("/content") else pathlib.Path.cwd()
path = kagglehub.dataset_download("rupankarmajumdar/crop-pests-dataset")

local_path = pathlib.Path("/content/datasets/crop-pests")
local_path.parent.mkdir(parents=True, exist_ok=True)
shutil.copytree(path, local_path, dirs_exist_ok=True)

data_yaml_path = local_path / "data.yaml"
data_cfg = {
    "path": str(local_path),
    "train": "train/images",
    "val":   "valid/images",
    "test":  "test/images",
    "names":
     [
      "Ants",
      "Bees",
      "Beetles",
      "Catterpillars",
      "Earthworms",
      "Earwigs",
      "Grasshoppers",
      "Moths",
      "Slugs",
      "Snails",
      "Wasps",
      "Weevils",
    ]
}

with open(data_yaml_path, "w") as f:
    yaml.safe_dump(data_cfg, f)
#**********************************************
# YOLO model training
yolo = YOLO("yolov8n.pt")
yolo.train(
    data=str(data_yaml_path),
    epochs=15,
    imgsz=640,
    batch=8,
    device=0,
    workers=2,
    project=str(base_p / "yolo_runs"),
    name="yolo_effnet",
    verbose=False,
)
#**********************************************
# Crop images
def extract_crop_from_box(img, cid, xc, yc, bw, bh):

    h, w = img.shape[:2]
    # Convert normalized YOLO coords to pixel coords
    cx = xc * w
    cy = yc * h
    bw_px = bw * w
    bh_px = bh * h

    # Box corners
    x1 = int(max(0, cx - bw_px / 2))
    y1 = int(max(0, cy - bh_px / 2))
    x2 = int(min(w, cx + bw_px / 2))
    y2 = int(min(h, cy + bh_px / 2))

    crop = img[y1:y2, x1:x2]
    if crop.size == 0:
        return None

    crop = cv2.resize(crop, (300, 300))

    cname = data_cfg["names"][cid]
    return crop, cname

#**********************************************

def make_crops(split_rel, split_name):
    img_dir = local_path / split_rel
    lab_dir = img_dir.parent / "labels"

    out_root = local_path / "cls_ds" / split_name
    out_root.mkdir(parents=True, exist_ok=True)

    img_paths = (
        list(img_dir.glob("*.jpg"))
        + list(img_dir.glob("*.png"))
        + list(img_dir.glob("*.jpeg"))
    )

    k = 0
    for p in img_paths:
        lp = lab_dir / (p.stem + ".txt")
        if not lp.exists():
            continue

        img = cv2.imread(str(p))
        if img is None:
            continue

        with open(lp, "r") as f:
            for line in f:
                parts = line.strip().split()
                if not parts:
                    continue

                cid = int(parts[0])
                if cid < 0 or cid >= len(data_cfg["names"]):
                    continue

                xc, yc, bw, bh = map(float, parts[1:])

                out = extract_crop_from_box(img, cid, xc, yc, bw, bh)
                if out is None:
                    continue

                crop, cname = out

                cls_dir = out_root / cname
                cls_dir.mkdir(parents=True, exist_ok=True)

                op = cls_dir / f"{p.stem}_{k}.jpg"
                cv2.imwrite(str(op), crop)
                k += 1

tr_img = "train/images"
va_img = "valid/images"

make_crops(tr_img, "train")
make_crops(va_img, "val")

cls_root = local_path / "cls_ds"

#**********************************************
# load data
tr_tf = T.Compose([
    T.Resize((300, 300)),
    T.RandomHorizontalFlip(),
    T.RandomRotation(10),
    T.ColorJitter(brightness=0.2, contrast=0.2),
    T.ToTensor(),
])

va_tf = T.Compose([
    T.Resize((300, 300)),
    T.ToTensor(),
])

tr_ds = ImageFolder(str(cls_root / "train"), transform=tr_tf)
va_ds = ImageFolder(str(cls_root / "val"), transform=va_tf)

tr_loader = DataLoader(
    tr_ds,
    batch_size=32,
    shuffle=True,
    num_workers=2,
    pin_memory=True,
)
va_loader = DataLoader(
    va_ds,
    batch_size=32,
    shuffle=False,
    num_workers=2,
    pin_memory=True,
)

n_cls = len(tr_ds.classes)
#**********************************************
#**********************************************
# EfficientNet-B3 Classifier
net = EfficientNet.from_pretrained("efficientnet-b3")
fc_in = net._fc.in_features
net._fc = nn.Linear(fc_in, n_cls)
net = net.cuda()

crit = nn.CrossEntropyLoss()
opt = optim.Adam(net.parameters(), lr=1e-4)

#**********************************************
def train_one_epoch(m, tr_l):
    m.train()
    s_loss = 0.0
    s_corr = 0
    s_tot = 0

    for x, y in tr_l:
        x = x.cuda()
        y = y.cuda()

        opt.zero_grad()
        out = m(x)
        loss = crit(out, y)
        loss.backward()
        opt.step()

        s_loss += loss.item() * x.size(0)
        _, pred = out.max(1)
        s_corr += pred.eq(y).sum().item()
        s_tot += y.size(0)

    tr_loss = s_loss / s_tot
    tr_acc = s_corr / s_tot
    return tr_loss, tr_acc
#**********************************************

def eval_one_epoch(m, va_l):
    m.eval()
    v_loss_sum = 0.0
    v_corr = 0
    v_tot = 0

    with torch.no_grad():
        for x, y in va_l:
            x = x.cuda()
            y = y.cuda()
            out = m(x)
            loss = crit(out, y)
            v_loss_sum += loss.item() * x.size(0)
            _, pred = out.max(1)
            v_corr += pred.eq(y).sum().item()
            v_tot += y.size(0)

    v_loss = v_loss_sum / v_tot
    v_acc = v_corr / v_tot
    return v_loss, v_acc
#**********************************************

def train_net(m, tr_l, va_l, epochs=5):
    best_acc = 0.0
    best_w = None

    for ep in range(1, epochs + 1):
        tr_loss, tr_acc = train_one_epoch(m, tr_l)
        v_loss, v_acc = eval_one_epoch(m, va_l)

        print(ep, round(tr_loss, 4), round(tr_acc, 4), round(v_loss, 4), round(v_acc, 4))

        if v_acc > best_acc:
            best_acc = v_acc
            best_w = m.state_dict()

    if best_w is not None:
        m.load_state_dict(best_w)
    return m

net = train_net(net, tr_loader, va_loader, epochs=5)

net_p = base_p / "eff_b3_cp.pt"
torch.save(net.state_dict(), net_p)
#**********************************************
#**********************************************
# YOLO + EffNet
yolo.eval()

net = EfficientNet.from_pretrained("efficientnet-b3")
fc_in = net._fc.in_features
net._fc = nn.Linear(fc_in, n_cls)
net.load_state_dict(torch.load(net_p, map_location="cuda"))
net = net.cuda()
net.eval()

inf_tf = T.Compose([
    T.Resize((300, 300)),
    T.ToTensor(),
])

idx2cls = {i: c for i, c in enumerate(tr_ds.classes)}

#**********************************************
#**********************************************
def apply_size_prior(probs, area_pct):
    w = probs.clone()

    if area_pct < 0.005:  # tiny box
        for c in big_cls:
            if c in cls2idx:
                w[cls2idx[c]] *= 0.5

    if area_pct > 0.05:  # big box
        for c in small_cls:
            if c in cls2idx:
                w[cls2idx[c]] *= 0.5

    s = w.sum()
    if s > 0:
        w = w / s
    return w
#**********************************************

def pred_one(pth, th=0.25):
    img_bgr = cv2.imread(pth)
    if img_bgr is None:
        raise ValueError("bad img " + str(pth))
    img_rgb = cv2.cvtColor(img_bgr, cv2.COLOR_BGR2RGB)

    h_img, w_img = img_bgr.shape[:2]
    r = yolo(img_rgb, imgsz=640)[0]
    out = []

    for b in r.boxes:
        d_conf = float(b.conf.cpu().item())
        if d_conf < th:
            continue

        x1, y1, x2, y2 = b.xyxy[0].cpu().numpy().astype(int)
        x1 = max(0, min(x1, w_img - 1))
        x2 = max(0, min(x2, w_img - 1))
        y1 = max(0, min(y1, h_img - 1))
        y2 = max(0, min(y2, h_img - 1))
        if x2 <= x1 or y2 <= y1:
            continue

        box_w = x2 - x1
        box_h = y2 - y1
        area_pct = (box_w * box_h) / float(w_img * h_img + 1e-6)

        crop_bgr = img_bgr[y1:y2, x1:x2]
        if crop_bgr.size == 0:
            continue

        crop_rgb = cv2.cvtColor(crop_bgr, cv2.COLOR_BGR2RGB)
        crop_pil = T.functional.to_pil_image(crop_rgb)
        inp = inf_tf(crop_pil).unsqueeze(0).cuda()

        with torch.no_grad():
            logits = net(inp)
            probs = torch.softmax(logits, dim=1)[0]
            cid = int(torch.argmax(probs).item())
            c_conf = float(probs[cid].item())
            cname = idx2cls[cid]

        out.append({
            "bbox": [int(x1), int(y1), int(x2), int(y2)],
            "cls": cname,
            "det_conf": d_conf,
            "cls_conf": c_conf,
        })

    return out
#**********************************************

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 48.0 MB/s eta 0:00:00
Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
Using Colab cache for faster access to the 'crop-pests-dataset' dataset.
Ultralytics 8.3.228 🚀 Python-3.12.12 torch-2.8.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=8, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/datasets/cp_yolo_eff/data.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None,

100%|██████████| 47.1M/47.1M [00:01<00:00, 32.1MB/s]

Loaded pretrained weights for efficientnet-b3


1 0.6788 0.8194 0.3439 0.9016
2 0.1582 0.9545 0.3317 0.9128
3 0.0784 0.9775 0.3599 0.9135
4 0.0469 0.9872 0.4031 0.906
5 0.0352 0.99 0.4003 0.912
Loaded pretrained weights for efficientnet-b3
